# 🎯 Chess AI Training on Google Colab
## ใช้ GPU ฟรีเพื่อ train AI หมากรุก

**วิธีใช้:**
1. Runtime → Change runtime type → GPU
2. รัน cells ทั้งหมดตามลำดับ

In [ ]:
# Cell 1: ติดตั้ง Dependencies
!pip install python-chess tqdm -q
print('✅ Dependencies installed!')

In [ ]:
# Cell 2: Import Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import chess
import random
import numpy as np
from collections import deque
from tqdm import tqdm

print(f'PyTorch: {torch.__version__}')
print(f'CUDA: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'\n🖥️ Using: {device}')

In [ ]:
# Cell 3: Neural Network Model
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(channels)
        self.conv2 = nn.Conv2d(channels, channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(channels)
    
    def forward(self, x):
        r = x
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        return F.relu(x + r)

class ChessNet(nn.Module):
    def __init__(self, blocks=10, channels=128):
        super().__init__()
        self.conv = nn.Conv2d(12, channels, 3, padding=1)
        self.bn = nn.BatchNorm2d(channels)
        self.res = nn.ModuleList([ResidualBlock(channels) for _ in range(blocks)])
        self.pc = nn.Conv2d(channels, 32, 1)
        self.pbn = nn.BatchNorm2d(32)
        self.pfc = nn.Linear(32*64, 4672)
        self.vc = nn.Conv2d(channels, 1, 1)
        self.vbn = nn.BatchNorm2d(1)
        self.vfc1 = nn.Linear(64, 256)
        self.vfc2 = nn.Linear(256, 1)
    
    def forward(self, x):
        x = F.relu(self.bn(self.conv(x)))
        for b in self.res: x = b(x)
        p = F.relu(self.pbn(self.pc(x))).view(-1, 32*64)
        p = self.pfc(p)
        v = F.relu(self.vbn(self.vc(x))).view(-1, 64)
        v = torch.tanh(self.vfc2(F.relu(self.vfc1(v))))
        return p, v

print('✅ Model defined!')

In [ ]:
# Cell 4: Helper Functions
def board_to_tensor(board):
    t = np.zeros((12, 8, 8), dtype=np.float32)
    pm = {(chess.PAWN,True):0,(chess.KNIGHT,True):1,(chess.BISHOP,True):2,
          (chess.ROOK,True):3,(chess.QUEEN,True):4,(chess.KING,True):5,
          (chess.PAWN,False):6,(chess.KNIGHT,False):7,(chess.BISHOP,False):8,
          (chess.ROOK,False):9,(chess.QUEEN,False):10,(chess.KING,False):11}
    for sq in chess.SQUARES:
        p = board.piece_at(sq)
        if p: t[pm[(p.piece_type, p.color)], sq//8, sq%8] = 1.0
    return t

class AI:
    def __init__(self, model, explore=0.2):
        self.model, self.explore = model, explore
        self.dev = next(model.parameters()).device
    
    def get_move(self, board):
        moves = list(board.legal_moves)
        if not moves: return None
        if random.random() < self.explore: return random.choice(moves)
        self.model.eval()
        best, bv = None, float('-inf')
        for m in moves:
            board.push(m)
            s = torch.from_numpy(board_to_tensor(board)).unsqueeze(0).to(self.dev)
            with torch.no_grad(): _, v = self.model(s)
            board.pop()
            if -v.item() > bv: bv, best = -v.item(), m
        return best or random.choice(moves)

print('✅ Helpers ready!')

In [ ]:
# Cell 5: Training System
class Trainer:
    def __init__(self, blocks=10, channels=128):
        self.model = ChessNet(blocks, channels).to(device)
        self.opt = optim.Adam(self.model.parameters(), lr=0.001)
        self.buf = deque(maxlen=100000)
        self.stats = {'games':0, 'w':0, 'b':0, 'd':0}
        print(f'Parameters: {sum(p.numel() for p in self.model.parameters()):,}')
    
    def play(self, max_moves=150):
        board = chess.Board()
        data = []
        ai = AI(self.model, 0.3)
        while not board.is_game_over() and board.fullmove_number <= max_moves:
            data.append({'s': board_to_tensor(board), 't': board.turn})
            m = ai.get_move(board)
            if not m: break
            board.push(m)
        
        if board.is_checkmate():
            r = -1 if board.turn else 1
            if board.turn: self.stats['b'] += 1
            else: self.stats['w'] += 1
        else:
            r = 0
            self.stats['d'] += 1
        
        for d in data:
            v = r if d['t'] else -r
            self.buf.append((torch.from_numpy(d['s']), v))
        
        self.stats['games'] += 1
        return r, board.fullmove_number
    
    def train_batch(self, bs=128):
        if len(self.buf) < bs: return 0
        self.model.train()
        batch = random.sample(self.buf, bs)
        s, v = zip(*batch)
        s = torch.stack(s).to(device)
        v = torch.tensor(v, dtype=torch.float32).to(device).unsqueeze(1)
        self.opt.zero_grad()
        _, vo = self.model(s)
        loss = nn.MSELoss()(vo, v)
        loss.backward()
        self.opt.step()
        return loss.item()
    
    def run(self, games=1000, save_every=200):
        print(f'\n🎮 Training {games} games on {device}...')
        for g in tqdm(range(1, games+1)):
            self.play()
            if g % 10 == 0: self.train_batch()
            if g % save_every == 0:
                torch.save(self.model.state_dict(), f'/content/model_{g}.pt')
        
        torch.save(self.model.state_dict(), '/content/chess_model_final.pt')
        print(f'\n✅ Done! W:{self.stats["w"]} B:{self.stats["b"]} D:{self.stats["d"]}')

print('✅ Trainer ready!')

In [ ]:
# Cell 6: 🚀 RUN TRAINING
trainer = Trainer(blocks=10, channels=128)
trainer.run(games=1000, save_every=200)

In [ ]:
# Cell 7: 📥 Download Model
from google.colab import files
files.download('/content/chess_model_final.pt')
print('✅ Model downloaded!')